# Computer lab 1 (CL1) - Linear regression

The *python-crash-course* taught you general basics of python.
Now the computer labs *CL1* and *CL2* will specifically introduce you to python for machine learning.
The goal is to provide you with a few hands-on examples of simple regression and classification so that you can build intuition for these types of problems and to show you a standard way of developing a machine learning algorithm in python.

The labs will give you experience with a few well-known python libraries (called *modules*) which will be used in the course.
Extra important is the machine learning module called `pytorch`, but also libraries for data manipulation and visualisation, such as `pandas`, `matplotlib` and `numpy`.

We strongly encourage you to play with the code and explore ways of manipulating and plotting the data, that is how you build up your intuition.

For this computer lab, we'll be using the IRIS dataset. Initially, we'll only look at a subset of it, and perform linear regression on two features of a given class.

# 1. Loading the data

### 1.1  Import the necessary modules

We'll use these three different modules, and one of the functions from scikit-learn.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
# This last line is a special jupyter command
# which makes matplotlib plots show up in notebooks themselves.

### 1.2  Read the dataset from a .csv file

Load the [IRIS dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set) using `pandas`. The method `read_csv(<filename>)` takes a path to a csv-file and returns a `DataFrame` object containing the data found in the file.

`pandas` data frames are a great way of handling simple data, where the entire dataset can be read into the computer memory all at once.
Later, we will handle more complex data which will require us to create so called *Dataloaders*.

In [ ]:
# The file `iris.csv` is located in our current directory (<repo_root>/computer-labs/CL1),
# otherwise the function would fail.
dataset = pd.read_csv("iris.csv")

In [ ]:
type(dataset)

### 1.3  Analyze the dataset

This dataset is comprised of morphologic data from three different species of the Iris flowers: Setosa, Virginica and Versicolor.

<table style="width:100%">
  <tr>
    <th> <center>Iris Setosa</center> </th>
    <th> <center>Iris Virginica</center> </th> 
    <th> <center>Iris Versicolor</center> </th>
  </tr>
  <tr>
    <td><img src="https://upload.wikimedia.org/wikipedia/commons/5/56/Kosaciec_szczecinkowaty_Iris_setosa.jpg" alt="Iris Setosa"></td>
    <td><img src="https://upload.wikimedia.org/wikipedia/commons/9/9f/Iris_virginica.jpg" alt="Iris Virginica"></td>
    <td><img src="https://upload.wikimedia.org/wikipedia/commons/2/27/Blue_Flag%2C_Ottawa.jpg" alt="Iris Virginica"></td>
  </tr>
</table>

The lenght and width of both the petals and the sepals of each flower, together with its corresponding species were measured and stored in this dataset. Sepals and petals are both parts of a flower. Sepals are the outermost part of the whorl and the petals are the innermost part.
![](http://terpconnect.umd.edu/~petersd/666/html/iris_with_labels.jpg)

Let's take a look at what's inside the dataset now. The attribute `shape` of `DataFrame` objects returns the dimensions of the data inside it.

In [ ]:
dataset.shape

So this dataset has 150 rows and 5 columns. It's easy to infer that this means 150 flowers were collected, and 5 different features were registered for each one. We can also take a closer look at them, using the method `head()`, which returns the first 5 rows by default (you can also pass a parameter to it, which specifies a different amount of rows to be shown).

In [ ]:
dataset.head()

Here we can see the header names for each column, together with the first rows, confirming that the species and morphologic measurements for each flower were collected. We can extract individual columns of this `DataFrame` by indexing using their names, for instance:

In [ ]:
dataset["sepal_length"]

Additionally, we can check which species are present in the dataset using the `unique` method,

In [ ]:
print(dataset["species"].unique())

where we see that only these three species are present in this dataset, as expected.

We can also learn more about the data types of each column with the method `info`.

In [ ]:
dataset.info()

Here we see that the first four columns' elements are floating point numbers, and the last column's elements are objects (in this case, strings).

### 1.4  Extract the desired data

For this initial task, we are only interested in the setosa species. This corresponds to all the rows which have the column 'species' equal to the string 'setosa'. In order to extract these rows, we use [logical indexing in Pandas](https://pandas.pydata.org/pandas-docs/stable/indexing.html#boolean-indexing).

In [ ]:
# This returns a boolean series, which for every row in the dataframe
# if a row is a setosa row or not.
# We can use it to index our DataFrame object
extract_rule = (dataset['species']=='setosa')
print(extract_rule)

In [ ]:
# We use the boolean series to index the DataFrame object.
# This will give us a new dataframe which only contains the setosa rows
setosa_dataset = dataset[extract_rule]
setosa_dataset

We can also extract columns from the `DataFrame`.
Suppose that we want to investigate the relationship between two features of this species, the 'sepal_length' and 'sepal_width'. To extract these, we [index the `DataFrame` using the name of the columns](https://pandas.pydata.org/pandas-docs/stable/indexing.html#selection-by-label)  we want.

In [ ]:
x = setosa_dataset['sepal_length'].values
y = setosa_dataset['sepal_width'].values

Note that the attribute `values` in a `DataFrame` object returns a numpy array.
That is, extracting a single column in a `DataFrame` gives us a numpy array, not another `DataFrame`

In [ ]:
type(x)

Now we can use matplotlib to plot all the examples in a 2D plane, where each dimension is one of the features described earlier.

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x,y)
ax.set_xlabel('sepal length')
ax.set_ylabel('sepal width');

It seems like the relation between these features could be approximated using a linear function, such as 
$f(x) = w\cdot x + b$. Let's try finding the parameters $w$ and $b$ that would make the best approximation.

### 1.5  Guess the values of w and b

We'll start with some educated guesses. To make this more convenient, we'll first define a function to plot a scatter plot of the provided data, together with a straight line with parameters specified by the user.

In [ ]:
# Define a function to plot the data and a parameterized line
def plot_data_and_line(w, b, x, y, ax, line_color='r', line_label=''):
    
    # Create points lying on the line
    xline = np.unique(x)
    yline = w*xline + b

    # Plot both the line and the points from the dataset
    ax.scatter(x,y, color='C0')
    ax.plot(xline, yline, color=line_color, label=line_label)
    ax.set_xlabel('sepal length')
    ax.set_ylabel('sepal width') 

In [ ]:
fig, ax = plt.subplots()
plot_data_and_line(1, -1, x, y, ax)

Additionally, another way of evaluating the quality of our approximation is to compute the MSE ([mean squared error](https://www.freecodecamp.org/news/machine-learning-mean-squared-error-regression-line-c7dde9a26b93/)) between the true y features in the dataset and our predictions. So that we can use this value as well to guide our guesses, create a function to compute it (first, it might be beneficial to write down the analytical expression for it).

In [ ]:
# Create a function called `mse` to compute the mean squared error
# YOUR CODE HERE

Now we can try different values of $w$ and $b$ and see how the resulting linear approximation looks like, compared to the scatter plot of our data. Using both the plot and the MSE, try searching for values of $w$ and $b$ that yield a good approximation.

In [ ]:
# Here we just check that you actually implemented the `mse` function. 
try:
    mse
except NameError:
    raise NotImplementedError(
        "You need to implement a function `mse` in the cell above. Don't forget to run the cell!")

# Guess the values for w and b

# YOUR CODE HERE

# Plot your guess
plt.close('all')
fig, ax = plt.subplots()
plot_data_and_line(w, b, x, y, ax);

# Compute guess

y_guess = w*x+b

# Some numpy operations can change the dimensions of the output.
# e.g. it changes an array from shape (50, 1) to (50,)
# to us this means the same thing, it's an object with 50 elements.
# However, it can mess with later computations. To be sure we can reshape `y_guess`
# to force it to be of the same shape as `y`.
# (w*x+b should not do it but we'll use it as an example anyway.)
y_guess = y_guess.reshape(y.shape)

# Compute and print the MSE of the guess
print("MSE of your guess:", mse(y,y_guess))

# Sanity check: the MSE for the guess (w, b) = (1, -1) should be around 0.41

---

# 2. Training a model for linear regression

Now, instead of trying to find the parameters that give the best approximation by trial and error, we'll use `pytorch` to build and optimize a linear regressor neural network.

`pytorch` is a widely used library or *module* for doing machine learning in python.
It allows user to construct, train and evaluate general neural networks.
A more thorough introduction of `pytorch` comes later in the labs.

It is not necessary for completing these labs, but if you are interested, take a look at `pytorch`'s
- [documentation](https://pytorch.org/docs/stable/index.html)
- [tutorials](https://pytorch.org/tutorials/)


### 2.1 Create dataset and data loaders from the data

`pytorch` expects to get data from a so called *data loader*.
A data loader is an object that provides your machine learning model with *batches* of data.
That way, we don't have to load our entire dataset into memory at once. Instead we get smaller, more manageable batches of data to work with.
To create a `DataLoader` object, we first create a `TensorDataset`. The actual dataset can be stored in memory, on the hard drive or even on some remote server. `TensorDataset` is an abstraction over our data which provides methods for how to obtain a sample from our data.
Finally, the `DataLoader` class takes an instance of our `TensorDataset` class (and some other configuration parameters) and automatically provides methods for iterating over the full data in batches.

It is a bit overkill to do this with our simple Iris data (we have already seen how to do it with a `DataFrame`),
but later in the course we will use more complex data which do require a dataloader.

In [ ]:
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

Current shape of `x` (and `y`) is (50,).

In [ ]:
print("Shape of x: {}".format(x.shape))
print("Shape of y: {}".format(y.shape))

Again, we need to be careful with the actual dimension of our dataset.

For the class `TensorDataset` to work properly, we need to add a second dimension to `x`.
The loader expects data of size `(N,D)`, where `N` is the number of samples in the dataset, and `D` is the dimension of each sample.
So for our example we have:

N = 50, D = 1

The dimension of `y` can be `(50,)` because we won't actually pass it through the model.
We might however need to later check so that this shape works with our loss function.

In [ ]:
# A single dimension can be added in multiple ways.
# It is good practice to use the `.reshape(<new_shape>)` function
# since it clearly communicates what we want the new shape to be.

N = x.shape[0] # x.shape is a tuple (N, ) where the first (0th) element is N
x = x.reshape((N, 1))
print("The new shape of x is {}".format(x.shape))

Now we create the dataset and the data loader.

Note that in the data loader we specify the batch size to match the size of the dataset.
Typically, we will use batch sizes which are much smaller than the total number of samples.

In [ ]:
dataset = TensorDataset(torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32))
data_loader = DataLoader(dataset, batch_size=len(x))

### 2.2 Create the model for linear regression.

Now we actually create our model.

We define a class `LinearRegressor` which inherits from `nn.Module`.
`nn.Module` is `pytorch`'s basic neural network class.

In [ ]:
from torch import nn

In [ ]:
class LinearRegressor(nn.Module):
    def __init__(self):
        """The __init__ function creates our model
        by creating a network which only has a single linear layer.
        """
        super().__init__()
        self.lin = nn.Linear(1, 1)

    def forward(self, x):
        """Every `pytorch` model has a forward method.
        It describes how a network, or even a single layer transforms an input `x` to an output
        
        Here, we simple refer the input to our linear layer.
        """
        return self.lin(x)

# Create an instance of the model class.
model = LinearRegressor()

# `model` is a simple linear transformation. By default its parameters (w and b) are intialised randomly,
# but we can still see how it works.

# Let's create some made-up input data.
# We need to jump through some hoops to make sure that the input is
# - of type torch.Tensor
# - with shape (1, 1)
test_x = torch.tensor(2.4).reshape((1, 1))

# Now we can use our model to predict y.
# Note: model(x) is just a shortcut for model.forward(x).
test_y = model(test_x)

# y will change with every run, since the model parameters are randomly chosen.
print("model({:.2f}) = {:.2f}".format(test_x.item(), test_y.item()))

### 2.3 Define loss function and optimizer

We use the mean squared error loss.
Although we have already implemented one above, we use the one built in to `pytorch`,
so that we can train our model with it.

In [ ]:
loss_fn = nn.MSELoss()

For optimization, we use stochastic gradient descent (since our batch size is the size of the dataset, we're actually doing gradient descent):

In [ ]:
from torch import optim
optimizer = optim.SGD(model.parameters(), lr=0.01)

### 2.4 Perform the optimization

Now we can perform the optimization by simply following these steps in a loop:
1. Sample a batch of data from our dataset
2. Compute the model's prediction on the batch
3. Compute the loss of the prediction w.r.t. ground-truth
4. Backpropagate the loss through the model's parameters
5. Perform one step of gradient descent.

We will do this for 20 epochs. Again, since our batch size is the same size of the dataset, this means we'll take 20 steps of gradient descent.

In [ ]:
for epoch in range(20):
    losses_in_epoch = []
    for batch in data_loader:

        # 1. These are the sampled batches of inputs and ground-truth
        batch_x, batch_y = batch
        
        # 2. Compute the model's prediction on the batch
        pred = model(batch_x)
        
        # 3. Compute the loss of the prediction w.r.t. ground-truth
        loss = loss_fn(pred.squeeze(), batch_y)
        
        # Save losses in a list for averaging later (not sctrictly necessary for batch_size = len(x))
        losses_in_epoch.append(loss)
        
        # 4. Backpropagation
        loss.backward()
        
        # 5. One step of gradient descent
        optimizer.step()
        
        # Zero the gradients computed in the backpropagation, for starting new optimization step
        optimizer.zero_grad()
    
    print('Epoch: {}\tLoss: {}'.format(epoch, sum(losses_in_epoch)/len(losses_in_epoch)))

Note the final MSE obtained (~0.06). Compare it to the one obtained using the guessed parameters. 

### 2.5  Extract parameters w and b

Extract the parameters found by the optimization by using the `parameters` method of the created model (this returns a [generator](https://www.programiz.com/python-programming/generator), so we transform it into a list first).

In [ ]:
parameters = list(model.parameters())

Each element in this list is a `Parameter` object:

In [ ]:
parameters

We can access the underlying tensor by using the `data` attribute of the `Parameter` class:

In [ ]:
parameters[0].data

And the float inside the tensor with the `item` method (only works with one-element tensors).

In [ ]:
parameters[0].data.item()

Putting this together we can get the parameters of our model as follows:

In [ ]:
w_star, b_star = [p.data.item() for p in parameters]

Which results in:

In [ ]:
print("w*: {:.3f}".format(w_star))
print("b*: {:.3f}".format(b_star))

Compare these optimized parameters with the ones you guessed before.

### 2.5  Compare optimal and guessed values

Finally, it's also beneficial to compare the guessed parameters with the optimized ones graphically, by showing both of the predicted lines in the same plot.

In [ ]:
plt.close('all')
fig, ax = plt.subplots()
plot_data_and_line(w, b, x, y, ax, 'r', 'guess')
#plot_data_and_line(w_star, b_star, x, y, ax, 'b', 'optimal')
ax.legend();